The following notebook was adapted from Angenent-Mari et al. Their original code can be found here: https://github.com/lrsoenksen/CL_RNA_SynthBio. 

Load packages 

In [48]:
## Import Libraries
# General system libraries
import gc
import numpy as np
import pandas as pd

# Math & Visualization Libs
import matplotlib.pyplot as plt
from scipy import stats

# Import Tensorflow
import tensorflow as tf

# Import Keras
from keras import backend as K
from keras.layers import BatchNormalization, Dropout, Flatten, Dense, Input #, Activation, Conv1D, Conv2D, Reshape, Lambda, InputLayer, Convolution2D, MaxPooling1D, MaxPooling2D, ZeroPadding2D, Bidirectional
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Import sklearn libs
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

# Progress Bar
from tqdm import tqdm

# Warnings
import warnings
warnings.filterwarnings("ignore")

Training and cross-validating model based on toehold switch data

In [49]:
# Define path to load desired Toehold dataset file (.csv)
data_filename = "data/mit-data.csv"
#data_filename = "Sensor-Data.csv"
#data_path = data_folder + data_filename
data = pd.read_csv(data_filename)

# Fix random seed for reproducibility
seed = 7
np.random.seed(seed) # Seed can be any number

In [50]:
#Creating a random column to test performance and replacing T's with U's in sequences to coorespond to ribosensor data
data['random'] = np.random.random(size=len(data))
data['seq_SwitchON_GFP'] = data['seq_SwitchON_GFP'].str.replace('T', 'U') 

In [ ]:
### Data Sequence ID selection
#id_data = data['sequence_id']

### Toehold Switch dataset input/output columns for selection
input_cols = 'seq_SwitchON_GFP'
output_cols = ['ON', 'OFF', 'ON_OFF','random']
qc_levels = [1.1]
doTrain = True
loss_init = 'mae' #'logcosh', #'mse', 'mae', 'r2'
n_foldCV = 5
verbose_init = True
evaluate  = True
display_init = False

### Define data scaler (if any)
scaler_init = False
scaler = QuantileTransformer(output_distribution='uniform')

### DEFINE MODEL NAME (e.g. MLP, CNN, LSTM, etc.)
model_name = 'MLP_1D'

#Show sample of dataframe structure
data.head()

In [52]:
# Helper function to pass string DNA/RNA sequence to one-hot
def dna2onehot(seq):
    #get sequence into an array
    seq_array = np.array(list(seq))
    
    #integer encode the sequence
    label_encoder = LabelEncoder()
    integer_encoded_seq = label_encoder.fit_transform(seq_array)
    
    #one hot the sequence
    onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    #reshape because that's what OneHotEncoder likes
    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
    onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)
    
    return onehot_encoded_seq

In [53]:
# INPUT / OUTPUT DEFINITION, PROCESSING & LOADING
#def pre_process_data (data, org_cols, input_cols, output_cols, scaler_init=False, display=True):
def pre_process_data (data, input_cols, output_cols, scaler_init=False, display=True):
    ## OUTPUT / INPUT DEFINITION, PROCESSING & LOADING
    
    #Init process bar
    tqdm.pandas() # Use `progress_apply` when `applying` one hot encoding and complementarity function to dataframe of input sequences

    df_data_input = data[input_cols].dropna()
    df_data_input = df_data_input.progress_apply(dna2onehot)
    data_input = np.array(list(df_data_input.values))
        
    # Data Output selection (QC filtered, OutColumns Only & Drop NaNs)
    df_data_output= data[output_cols]
    df_data_output = df_data_output.dropna(subset=output_cols)
    data_output = df_data_output.values.astype('float32')

    data_input = data_input[df_data_output.index.values][:][:]

    if scaler_init==True:
            data_output = scaler.fit_transform(data_output)
        
    # LOAD FULL LIST OF SEQUENCES after filtering
    data_seqs = data[input_cols][df_data_output.index.values]
        
    # Display processed data if desired
    if display==True:
        ### Show example of processed dataset
        ## Display number of retrieved sequences
        print("Number of sequences retrieved: "+str(len(data_input)))
        print()

        #Select ID to show
        toehold_id = 0 

        ## Plot Example input toehold matrix 
        print('EXAMPLE OF INPUT ONE-HOT TOEHOLD')
        onehot_encoded_seq = dna2onehot(data_seqs.iloc[toehold_id])
        print(onehot_encoded_seq.transpose())
        print()
        
        # Display input size
        print("Input Size: " + str(onehot_encoded_seq.shape))
        print()

        # Display example of Output vector
        print('EXAMPLE OF OUTPUT VECTOR')
        print(' ' + str(data_output[toehold_id]))
        print()

    return data_input, data_output

In [54]:
### Function to create Keras MLP for regression prediction
def create_mlp(width, height, regress=False):
    # Define our MLP network
    inputShape = (width, height) #DNA/RNA input sequence (one hot encoded)
    inputs = Input(shape=inputShape)
    chanDim = -1
    dropout_init = 0.2
    
    # Define dense layers
    x = inputs
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(dropout_init)(x)
    
    x = Dense(64, activation="relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(dropout_init)(x)

    x = Dense(32, activation="relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(dropout_init)(x)
    
    # Check to see if the regression node should be added
    if regress:
        x = Dense(len(output_cols), activation="linear")(x)
    else:
        x = Dense(len(output_cols), activation="sigmoid")(x)
        
    # Construct the Model
    model = Model(inputs, x) 
    
    # Return the model
    return model

In [55]:
#Definition of R2 metric for testing
def r2(x, y):
    return stats.pearsonr(x, y)[0] ** 2

#Definition of Custom metric as loss related to Coefficient of Determination (R2) 
#  CoD = 1 - MSE / (variance of inputs), and since this is going to be a loss we want 
#  improvement to point towards zero, so we choose mse/variance of inputs
def custom_r2_loss(y_true, y_pred): 
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return (SS_res/(SS_tot + K.epsilon()))

#Definition of Custom metric as loss related to Weigted Mean Absolute error
#  Improvement points towards zero, but penalizes loss for small values and improves it for larger values
def custom_wmae_loss(y_true, y_pred): 
    from keras import backend as K
    weightedMAE = K.abs((y_true-y_pred)*y_true) #Increase loss for large ON or OFF values -- Skews focus of distribution right
    return weightedMAE

In [56]:
#Definition of Coefficient of Determination (R2) for evaluation
def cod_r2(x,y):
    return r2_score(x,y)

In [57]:
### Define our final model architecture (layers & optimizor) and then compile it
def generate_model(trainX, testX, trainY, testY, verbose_init, evaluate=True):
    
    ## DEEP-LEARNING TRAINING PARAMETERS(e.g. verbose, patients, epoch size, batch size) to constrain process
    verbose_init = verbose_init #Zero is no keras verbose
    patience_init = 20 # Number of epochs to wait for no model improvement before early stopping a training
    epochs_init = 300 # Max number of epochs to perform (to cap training time)
    batch_size_init = 64*(1) # number of samples that will be propagated through the network at every epoch dependent on the number of GPUs
    validation_spit_init = 0.1 # Percentage of testing data to use in internal validation during training


    if verbose_init==True:
        # Callback to be used for checkpoint generation and early stopping
        callbacks_list = [EarlyStopping(monitor='val_loss', patience=patience_init, verbose = verbose_init)] # Adds Keras integration with TQDM progress bars.
    else:
        # Callback to be used for checkpoint generation and early stopping
        callbacks_list = [EarlyStopping(monitor='val_loss', patience=patience_init, verbose = False)]  
    
    ## Create Model (Change for MLP, CNN, ETC)
    # -------------------------------------------------------------------------------------------------------------------------------------
    # Define CNN model input shape
    (width, height) = np.shape(trainX[0])
    
    
    model = create_mlp(width, height, regress=False)

          
    ## Initialize the optimizer and Compile model:
    #   Custom metric is used (see above), if we use "Mean absolute percentage error" that
    #   implies that we seek to minimize the absolute percentage difference between 
    #   our *predictions* and *actual* output values. We also calculate other 
    #   valuable metrics for regression evaluation 
    opt = Adam(learning_rate=0.001, epsilon=1e-7, amsgrad=False) # epsilon=1e-1 for POISSON loss
    
    if loss_init=="r2":
        model.compile(loss=custom_r2_loss, optimizer=opt,  metrics=['mse','mae', 'mape', 'cosine_similarity', 'acc', custom_r2_loss])
    elif loss_init =="wmae":
        model.compile(loss=custom_wmae_loss, optimizer=opt,  metrics=['mse','mae', 'mape', 'cosine_similarity', 'acc', custom_wmae_loss])
    else:
        model.compile(loss=loss_init, optimizer=opt,  metrics=['mse','mae', 'mape', 'cosine_similarity', 'acc']) 


    model_history = model.fit(trainX, trainY, validation_split=validation_spit_init, epochs=epochs_init, batch_size=batch_size_init, callbacks=callbacks_list, verbose=verbose_init)
        
    # -------------------------------------------------------------------------------------------------------------------------------------  
        
       # Init prediction output matrix
    testX_Preds = np.zeros_like(testY) #Empty matrix for full prediction evaluation
    # Init performance metrics matrix
    test_metrics = np.zeros((trainY.shape[1],4)) #Empty matrix for model performance metrics
    #test_metrics = np.zeros((len(trainY),3))

    # GENERATE PREDICTIONS
    if testX.size > 0:
        ## Make predictions on testing data:
        print("Predicting functionality of Test Toeholds ...")
        print("")
        #Predictions in scaled space
        print(testX_Preds.shape)
        testX_Preds = model.predict(testX)
        print(testX_Preds.shape)
        
        if scaler_init == True:
            testY = scaler.inverse_transform(testY.reshape(-1, 1))
            testX_Preds = scaler.inverse_transform(testX_Preds)
        
        ## EVALUATE PERFORMANCE OF MODEL
        if evaluate==True:
            ## Plot training metrics per fold:
            plt.figure ()
            ax1 = plt.subplot(221)
            ax2 = plt.subplot(222)
            ax3 = plt.subplot(223)
            ax4 = plt.subplot(224)
            # Plot MSE metric
            ax1.set_title("Mean squared error")
            ax1.plot(model_history.history['mse'])
            # Plot MAE metric
            ax2.set_title("Mean absolute error")
            ax2.plot(model_history.history['mae'])
            # Plot MAPE metric
            ax3.set_title("Mean absolute percentage error")
            ax3.plot(model_history.history['mape'])
            # Plot CP metric
            ax4.set_title("Cosine Proximity")
            ax4.plot(model_history.history['cosine_similarity'])
            #Tight plot
            plt.tight_layout()
            

            ## Plot compiled training metrics per fold:
            plt.figure()
            plt.style.use("default")
            N = np.arange(0, len(model_history.history["loss"]))
            # Plot used Loss metric
            plt.plot(N, model_history.history["loss"], label="train_loss")
            plt.plot(N, model_history.history["val_loss"], label="test_loss")
            # Plot used Accuracy metric (applicable only if categorical model)
            plt.plot(N, model_history.history["acc"], label="train_acc")
            plt.plot(N, model_history.history["val_acc"], label="test_acc")
            # Plot MSE metric
            plt.plot(N, model_history.history["mse"], label="train_mse")
            plt.plot(N, model_history.history["val_mse"], label="test_mse")
            # Plot MAE metric
            plt.plot(N, model_history.history["mae"], label="train_mae")
            plt.plot(N, model_history.history["val_mae"], label="test_mae")
            # Plot MAPE metric
            plt.plot(N, model_history.history["mape"], label="train_mape")
            plt.plot(N, model_history.history["val_mape"], label="test_mape")
            # Plot CP metric
            plt.plot(N, model_history.history["cosine_similarity"], label="train_cp")
            plt.plot(N, model_history.history["val_cosine_similarity"], label="test_cp")
            plt.title("CNN Toehold Complementary Rep Data")
            plt.xlabel("Epoch #")
            plt.ylabel("Loss/Accuracy")
            plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.) # Place a legend to the right of this smaller subplot.
                        
            
            ## COMPUTE PERFORMANCE METRICS
            # Difference between the *predicted* toehold functionality values and *actual* toehold functionality values, 
            # then compute the absolute percentage difference for diplay
            diff = testX_Preds - testY
            abstDiff = np.abs(diff)
            # Compute the mean and standard deviation of the absolute difference:
            apd_mean = np.mean(abstDiff, axis=0)
            apd_std = np.std(abstDiff, axis=0)
            apd_r2 = np.zeros_like(apd_mean)
            apd_cod_r2 = np.zeros_like(apd_mean)
            # Plot: Predicted values vs. Experimental values and get R2 value
            for index,item in enumerate(output_cols):

                # R2 (Coefficient of Determination)
                apd_r2[index] = r2(testX_Preds[:,index], testY[:,index])
                apd_cod_r2[index] = cod_r2(testY[:,index], testX_Preds[:,index])
                
                # Display Output Values
                x=np.squeeze(testX_Preds[:,index])
                y=np.squeeze(testY[:,index])
                
                # Display Output Values
                print("" + item + " Mean_absolute_error (TEST): " + str(apd_mean[index]) + " (SD: " + str(apd_std[index]) + ")" )
                print('')
                print('EXPERIMENTAL Values vs. PREDICTED values (' + item + ')' )
                print('Pearson Correlation: '+ str(stats.pearsonr(x, y)[0]))
                print('Spearman Correlation: '+ str(stats.spearmanr(x, y)[0]))
                print('R^2: '+ str(apd_r2[index]))
                print('Coefficient of Determination (R2): ' + str(apd_cod_r2[index]))
                print('')
                
                # Store model performance metrics for return   
                test_metrics[index, :] = [apd_mean[index], apd_std[index], apd_r2[index], apd_cod_r2[index]]
                
                            
    return model, model_history, testX_Preds, test_metrics

In [58]:
### Define our crossvalidation model generator (layers, optimizor, compilation, training, reporting, etc)
def generate_crossval_model(X, Y, n_foldCV, verbose_init=True, evaluate=True):
    
    ## CROSSVALIDATION TRAINING
    # Define CV parameters
    n_foldCV = n_foldCV #Number of Crossvalidation bins
    cv_folds = list(StratifiedKFold(n_splits=n_foldCV, shuffle=True, random_state=seed).split(X,Y.argmax(1))) # Non repeating CV bins
    cv_preds = np.zeros_like(Y) #Empty matrix for full prediction evaluation
    cv_test_metrics = np.zeros((n_foldCV, Y.shape[1], 4))
    deploy_test_metrics = np.zeros((Y.shape[1],4))
    
    # Perform n-fold crossvalidated training and evaluation
    for j, (train_idx, test_idx) in enumerate(cv_folds):
        print('\nFold ',j)
        
        ## CrossValidation Strategy: 
        # We use all data for n-crossvalidation this will give us average metrics of performance in future data
        # for this all data will be devided into n bins. In every sequential fold we will use n-1 bins for training 
        # and the remaining bin for testing this split is done in such a way that all data is used for training and
        # testing at some point (sweet!). Testing points will be aggregated tenerate an average metric of performance
        # and all the datapoints will be put into a master agreement plot for visualization. 
        # A working model will be made using 75% of the data for training and 25% for further testing.
        # A deploy model will be also trained using all available data without testing
        trainX_cv = X[train_idx]
        trainY_cv = Y[train_idx]
        testX_cv = X[test_idx]
        testY_cv = Y[test_idx]
        # NOTE: Validation set is taken internally from the training set (10% of each fold), this is applied in the the model.fit function
        
        # Create & Train model each fold according to generator function
        model, model_history, testX_Preds, test_metrics = generate_model(trainX_cv, testX_cv, trainY_cv, testY_cv, verbose_init=verbose_init, evaluate=evaluate)
        
        # Record predicted values of each CV fold training to generate an ensemble reporting
        print("Predicting functionality of CV-Fold Test Toeholds & Model performance metrics ...")
        cv_preds[test_idx,:] = testX_Preds
        cv_test_metrics[j,:,:] = test_metrics
        
        ## MODEL MEMORY RELEASE
        del model_history
        del model
        for i in range(1): gc.collect()
        
        ## Free-up keras memmory to prevent leaks
        K.clear_session()
    
    #Transform back data
    if scaler_init == True:
        Y = scaler.inverse_transform(Y)
    
    # COMPUTE PERFORMANCE METRICS FOR DEPLOY MODEL
    # Difference between the *predicted* toehold functionality values and *actual* toehold functionality values, 
    # then compute the absolute percentage difference for diplay

    diff = cv_preds - Y
    abstDiff = np.abs(diff)
    # Compute the mean and standard deviation of the absolute difference:
    apd_mean = np.mean(abstDiff, axis=0)
    apd_std = np.std(abstDiff, axis=0)
    apd_r2 = np.zeros_like(apd_mean)
    apd_cod_r2 = np.zeros_like(apd_mean)

    ## EVALUATE ENSEMBLE CROSSVALIDATION PERFORMANCE OF MODEL
    if evaluate==True:
        for index,item in enumerate(output_cols): 
            # R^2 and Coefficient of Determination
            apd_r2[index] = r2(cv_preds[:,index], Y[:,index])
            apd_cod_r2[index] = cod_r2(Y[:,index], cv_preds[:,index])
            # Display Output Values
            x_tot=np.squeeze(cv_preds[:,index])
            y_tot=np.squeeze(Y[:,index])
            print('EXPERIMENTAL Values vs. PREDICTED values (' + item + ')' )
            print('Pearson Correlation: '+ str(stats.pearsonr(x_tot, y_tot)[0]))
            print('Spearman Correlation: '+ str(stats.spearmanr(x_tot, y_tot)[0]))
            print('R^2: '+ str(apd_r2[index]))
            print('Coefficient of Determination (R2): ' + str(apd_cod_r2[index]))
            print('')
            
            # Store model performance metrics for return   
            deploy_test_metrics[index, :] = [apd_mean[index], apd_std[index], apd_r2[index], apd_cod_r2[index]]
            
    ## DEPLOYMENT MODEL TRAINING (with full dataset)
    # Partition the data into training (80%), validation (10%), testing (10%) splits 
    (trainX, testX, trainY, testY) = train_test_split(X, Y, test_size=0.1, random_state=seed)
    
    # Create model function according to generator function
    model, model_history, _ , _ = generate_model(trainX, testX, trainY, testY, verbose_init=True, evaluate=True)
                                                                                                                                               
    ## Return                                     
    return model, model_history, cv_preds, cv_test_metrics, deploy_test_metrics

In [59]:
#Definer function for full model analysis and reporting
def execute_model_analysis(model_name, data, input_cols, output_cols, qc_levels, n_foldCV, verbose_init, evaluate):
      
    #Iterate through all desired Data QC levels
    for j, qc_level in enumerate(qc_levels): 
            
        ## LOAD PREPROCESSED INPUT / OUTPUT
        data_input, data_output = pre_process_data (data, input_cols, output_cols, scaler_init=scaler_init, display=display_init)
            
        ### 3) Model Training using Manual Verification Dataset & Evaluation
        # Training with a priori training (75%) & testing (25%) split, with internal training validation from the training set (10% or the 75%)
        # This also does valuation on unseen testing data (25%), and saves base model

        # Create manual model function according to generator function, train it and display architecture
        if doTrain==True:
            # Partition the data into training (75%) and testing (25%) splits
            (trainX, testX, trainY, testY) = train_test_split(data_input, data_output, test_size=0.25, random_state=seed)
            # Generate, Train, Evaluate, Save and Display Model
            model, model_history, testX_Preds, test_metrics = generate_model(trainX, testX, trainY, testY, verbose_init=verbose_init, evaluate=evaluate)
            model.summary()
        
            ## MODEL MEMORY RELEASE
            del model_history
            del model
            for i in range(1): gc.collect()
        
            ## Free-up keras memmory to prevent leaks
            K.clear_session()

        ### 4) Model Training using k-Fold Cross Validation, Ensemble Evaluation & Full Deployment
        # The gold standard for machine learning model evaluation is k-fold cross validation
        # It provides a robust estimate of the performance of a model on unseen data. 
        # It does this by splitting the training dataset into k subsets and takes turns training models on all subsets except one which is held out, and evaluating model performance on the held out validation dataset. The process is repeated until all subsets are given an opportunity to be the held out validation set. 
        # The performance measure accross all models in the unseen data for each fold
        # The performance is printed for each model and it is stored
        # A final deployment model trained in all data (no testing) is provided for evaluation in future data

        # Create crossvalidated model function according to generator function, train it and display architecture
        # Generate, Train, Evaluate, Save and Display Model
        if n_foldCV>0:
            model, model_history, cv_preds, cv_test_metrics, deploy_test_metrics = generate_crossval_model(data_input, data_output, n_foldCV=n_foldCV, verbose_init=verbose_init, evaluate=evaluate)
            model.summary()
            
            ## MODEL MEMORY RELEASE
            del model_history
            del model
            for i in range(1): gc.collect()
                
            ## Free-up keras memmory to prevent leaks
            K.clear_session()
    
   

In [ ]:
# RUN FULL MODEL ANALYSIS AND REPORTING model
execute_model_analysis(model_name, data, input_cols, output_cols, qc_levels, n_foldCV, verbose_init, evaluate)

Train on full toehold switch dataset and test on ribosensor data

In [61]:
# Define path to load desired Toehold dataset file (.csv)
data2_filename = "data/mit-data.csv"
data1_filename = "data/Sensor-Data-Update-9-11-23.csv"
#data_path = data_folder + data_filename
df1 = pd.read_csv(data1_filename)
df2 = pd.read_csv(data2_filename)

In [62]:
#Normalizing columns and targets between datasets
df2['seq_SwitchON_GFP'] = df2['seq_SwitchON_GFP'].str.replace('T', 'U')
df2 = df2[df2['seq_SwitchON_GFP'].notna()]
df1['SensorSequence'] = df1['SensorSequence'].str.replace(' ', '') 
df1['seq_SwitchON_GFP'] = df1['SensorSequence']
df1['ON_OFF'] = df1['FoldIncrease']/df1['FoldIncrease'].abs().max()
df1['random'] = np.random.random(size=len(df1))
df2['random'] = np.random.random(size=len(df2))
#df1.head()

In [63]:
### Dataset input/output columns for selection
org_cols = 'seq_SwitchON_GFP'
input_cols = 'pad'
output_cols = ['ON_OFF', 'random']
qc_levels = [1.1]
doTrain = True
loss_init = 'mae' #'logcosh', #'mse', 'mae', 'r2'
n_foldCV = 10
verbose_init = True
evaluate  = True
display_init = False

### Define data scaler (if any)
scaler_init = False
scaler = QuantileTransformer(output_distribution='uniform')

### DEFINE MODEL NAME (e.g. MLP, CNN, LSTM, etc.)
model_name = 'MLP_1D'

In [64]:
from data_prep import pad_sequences

In [65]:
# INPUT / OUTPUT DEFINITION, PROCESSING & LOADING
#def pre_process_data (data, org_cols, input_cols, output_cols, scaler_init=False, display=True):
def pre_process_data (data, input_cols, output_cols, scaler_init=False, display=True):
    ## OUTPUT / INPUT DEFINITION, PROCESSING & LOADING
    
    #Init process bar
    tqdm.pandas() # Use `progress_apply` when `applying` one hot encoding and complementarity function to dataframe of input sequences

    #Pad sequences to normalize length
    new_df = data.copy()
    pad_seq = pad_sequences(data[org_cols], pad_type='end', pad_length=190)
    new_df['pad'] = pad_seq

    df_data_input = new_df[input_cols].dropna()
    df_data_input = df_data_input.progress_apply(dna2onehot)
    data_input = np.array(list(df_data_input.values))
        
    # Data Output selection (QC filtered, OutColumns Only & Drop NaNs)
    df_data_output= data[output_cols]
    df_data_output = df_data_output.dropna(subset=output_cols)
    data_output = df_data_output.values.astype('float32')

    data_input = data_input[df_data_output.index.values][:][:]

    if scaler_init==True:
            data_output = scaler.fit_transform(data_output)
        
    # LOAD FULL LIST OF SEQUENCES after filtering
    data_seqs = new_df[input_cols][df_data_output.index.values]
    
    # Display processed data if desired
    if display==True:
        ### Show example of processed dataset
        ## Display number of retrieved sequences
        print("Number of sequences retrieved: "+str(len(data_input)))
        print()

        #Select ID to show
        toehold_id = 0 

        ## Plot Example input toehold matrix 
        print('EXAMPLE OF INPUT ONE-HOT TOEHOLD')
        onehot_encoded_seq = dna2onehot(data_seqs.iloc[toehold_id])
        print(onehot_encoded_seq.transpose())
        print()

        # Display input size
        print("Input Size: " + str(onehot_encoded_seq.shape))
        print()

        # Display example of Output vector
        print('EXAMPLE OF OUTPUT VECTOR')
        print(' ' + str(data_output[toehold_id]))
        print()

    return data_input, data_output

In [ ]:
#Generate model that trains on toehold switch data and tests on ribosensor data
trainX, trainY = pre_process_data (df2, input_cols, output_cols, scaler_init=False, display=False)
testX, testY = pre_process_data (df1, input_cols, output_cols, scaler_init=False, display=False)
model, model_history, testX_Preds, test_metrics = generate_model(trainX, testX, trainY, testY, verbose_init, evaluate=False)